# Author_Coauthor_Network


# The code contain three module
 1. Extraction of PubMed Data from NCBI's PubMed DB
 2. Calculat the Weight of the network
 3. Vizulaization of the coauthor Network
 

## Extraction Process

In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import json

API_KEY = "9446fd13127051ca7343c2018204779d8007"
keyword = "covid19"


esearch = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term="
#covid19&usehistory=true&webenv=true&retmax=10&api_key=9446fd13127051ca7343c2018204779d8007"

search_resp = requests.get(esearch+keyword+"&usehistory=true&webenv=true"+"api_key="+API_KEY)
search_soup = bs(search_resp.content,features="lxml")
webenv = search_soup.find("webenv").text
#count = search_soup.find("count").text
count =100
batch_size = 5
batches = []
for batch in range(0,int(count),5):
    efetch = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&query_key=1&webenv="    
    fetch_resp = requests.get(efetch+webenv+"&rettype=xml&api_key="+API_KEY+"&retmode=xml&retmax=2000&retstart="+str(batch))
    fetch_soup = bs(fetch_resp.content,features="lxml")
    articles_list = []
    PubmedArticleSet = fetch_soup.find("pubmedarticleset")
    articles = PubmedArticleSet.find_all("pubmedarticle")
    for article in articles:
        article_dict = {}
        title = article.find("articletitle").text        
        pmid = article.find("pmid").text    
        
        authors =  article.find("authorlist")
        author_list = []
        try:
            author_l = authors.find_all("author")
        except AttributeError:
            break        
            
        for auth in author_l:
            author_dict = {}
            try:
                first_name = auth.find("forename").text            
            except:
                first_name = ""            
            try:
                second_name = auth.find("lastname").text            
            except:
                second_name = ""           
            try:
                initials = auth.find("initials").text           
            except:
                initials = ""            
                
            author_name = initials+" " +first_name +" "+second_name            
            try:
                affiliation = auth.find("affiliation").text            
            except:
                affiliation = ""            
            author_dict["author_name"] = author_name            
            author_dict["affiliation"] = affiliation            
            author_list.append(author_dict)
        article_dict["title"] = title        
        article_dict["pmid"] = pmid        
        article_dict["author"] = author_list    
    batches.append(article_dict)
    print(batches)
    print("<------------------------------------------------------------------------------------------------>")

## Weight Calulation

In [ ]:
import pandas as pd
df = pd.DataFrame(batches)
df.columns = ['title', 'pmid', 'author']
df

In [ ]:
#ref:https://stackoverflow.com/questions/63875682/pandas-dataframe-splitting-a-column-with-dict-values-into-columns
#  explode the list
df = df.explode('author').reset_index(drop=True)

# now fill the NaN with an empty dict
df.author = df.author.fillna({i: {} for i in df.index})

# then normalize the column
df = df.join(pd.json_normalize(df.author))

# drop the column
df.drop(columns=['author'], inplace=True)
df

In [ ]:
def sort():
    b,n,k,l=[],[],[],[]
    x=0
    for i in range(0,5):
        if x==0:
            b.append(df['author_name'][i])
            n.append(df['affiliation'][i])
            t=df['pmid'][i]
            x+=1
        #print(x,t)
        elif x!=0 and t==df['pmid'][i]:
            k.append(df['author_name'][i])
            l.append(df['affiliation'][i])
        
    return b,n,k,l
b,n,k,l=sort()

In [ ]:
pm = df['pmid']
c = pd.DataFrame(b, columns=['First_Author'])
s = pd.DataFrame(n, columns=['First_Author\'s Affiliation'])
u = pd.DataFrame(k, columns=['Co-Author'])
q = pd.DataFrame(l, columns=['Co_Author\'s Affiliation'])
sep_author =pd.concat([pm,c,s,u,q],axis=1, join='inner')

In [ ]:
sep_author